In [140]:
import pandas as pd
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input

In [2]:
#Load raw
#raw to clean tweet
#pre process tweet
#one hot encoding stop, slow, restart


df = pd.read_csv('../../data/tweets_one_hot.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4807 entries, 0 to 4806
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     4807 non-null   object
 1   line     4807 non-null   object
 2   tweet    4807 non-null   object
 3   stop     4807 non-null   int64 
 4   restart  4807 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 187.9+ KB
None


In [85]:
def remove_short_word(tweet):
    tweet_ = []
    for word in tweet.split():
        if len(word) < 4:
            continue
            
        tweet_.append(word)

    return " ".join(tweet_)

#Remove english
#beaudry station reopened regular service resuming station
swe = stopwords.words('english')
swe_ = []
for word in swe:
    if len(word) < 4:
        continue
    
    swe_.append(word)

def sw_in(tweet):
    for word in tweet.split():
        if word in swe_:
            return True
        
    return False


In [197]:
df = pd.read_csv('../../data/tweets_one_hot.csv')
tweet = df.tweet

#REGEX and Character removal
regex = [
    (r"@\S*"),
    (r"#\w+"),
    (r"http\S*"),
    (r"\b\d+(\.\d+)?K\b"),
    (r"\d{1,2}h\d{2}"),
    (r"[\[\]()\/\\!?.,:;]"),
    "BLEUE",
    "VERTE",
    "JAUNE",
    "ORANGE",
    "show",
    "more",
    "this"
]
for r in regex:
    tweet = tweet.str.replace(r, "", regex=True)

#Lower case everything
tweet = tweet.str.lower()

#Remove short word
tweet = tweet.apply(remove_short_word)

#Remove english
tweet = tweet[~tweet.apply(sw_in)]

df['tweet_preprocess'] = tweet

#remove nan
df = df[df.tweet_preprocess.notna()]
df.tweet_preprocess.unique().size

1781

In [195]:
df.to_csv('../../data/tweets_one_hot_textProcess.csv')

# DEEP LEARNING

In [153]:
from gensim.models import Word2Vec

# Tokenize the tweets
tokenized_tweets = [tweet.split() for tweet in df.tweet.values]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_tweets, vector_size=32, window=5, min_count=1, workers=4)

# Example usage:
word_vectors = word2vec_model.wv
similar_words = word_vectors.most_similar('arrêt', topn=5)
print(similar_words)

[('prolongé', 0.9820728302001953), ('train', 0.9786366820335388), ('bonaventure', 0.9757501482963562), ('h-bourassa', 0.9742446541786194), ('beaubien', 0.9738559126853943)]


In [187]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming you have a list of tweets in `tweets`

# Convert the tweets into TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df.tweet)

# Apply K-means clustering
num_clusters = 20  # You can adjust this parameter
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(tfidf_matrix)

# Get the cluster labels for each tweet
cluster_labels = kmeans.labels_

# df_ = df.copy()
# df_['cluster'] = 0
# df_.cluster = cluster_labels
# df_[df_.cluster==3].tweet.unique()

# Print the tweets in each cluster
for cluster_id in range(num_clusters):
    print(f"Cluster {cluster_id}:")
    cluster_tweets = [df.tweet.iloc[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
    print(cluster_tweets[:5])  # Print the first 5 tweets in the cluster
    print()

Cluster 0:
['reprise graduelle service ligne', 'reprise graduelle service ligne', 'reprise graduelle service ligne', 'reprise graduelle service ligne', 'reprise graduelle service ligne']

Cluster 1:
['arrêt ligne entre snowdon st-michel personne autorisée voie reprise prévue vers', "arrêt ligne entre snowdon st-michel problème d'équipement électrique reprise prévue vers", "arrêt ligne entre snowdon st-michel inter services médicaux d'urgence reprise prévue vers", 'arrêt prolongé ligne entre snowdon st-michel incident durée indéterminée', 'arrêt ligne entre snowdon st-michel incident durée indéterminée']

Cluster 2:
['service normal métro', 'service normal métro ligne', 'service normal métro ligne', 'service normal métro ligne', 'service normal métro']

Cluster 3:
['arrêt ligne entre côte-vertu l-groulx dégagement fumée reprise prévue vers', 'arrêt ligne entre snowdon parc dégagement fumée reprise prévue vers', 'arrêt prolongé ligne entre côte-vertu l-groulx dégagement fumée reprise pré

In [134]:
max_words = 1000
max_len = 50
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df.tweet.values)
sequences = tokenizer.texts_to_sequences(df.tweet.values)
X_processed = pad_sequences(sequences, maxlen=max_len)

In [141]:
# Encoder
inputs = Input(shape=(max_len,))
encoded = Dense(128, activation='relu')(inputs)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

# Decoder
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(max_len, activation='sigmoid')(decoded)

# Autoencoder Model
autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 128)            │         6,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 50)             │         6,450 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,746 (131.82 KB)

 Trainable params: 33,746 (131.82 KB)

 Non-trainable params: 0 (0.00 B)

In [149]:
autoencoder.fit(word2vec_model, word2vec_model, epochs=10, batch_size=32, validation_split=0.2)

ValueError: Argument `validation_split` is only supported for tensors or NumPy arrays.Found incompatible type in the input: [<class 'gensim.models.word2vec.Word2Vec'>, <class 'gensim.models.word2vec.Word2Vec'>]

In [135]:
model = Sequential()
model.add(Embedding(max_words, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, df.stop.values, test_size=0.2, random_state=42)
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8180 - loss: 0.4995 - val_accuracy: 0.9882 - val_loss: 0.0568
Epoch 2/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9913 - loss: 0.0406 - val_accuracy: 0.9908 - val_loss: 0.0307
Epoch 3/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9959 - loss: 0.0164 - val_accuracy: 0.9908 - val_loss: 0.0301
Epoch 4/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9921 - loss: 0.0229 - val_accuracy: 0.9934 - val_loss: 0.0206
Epoch 5/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9989 - loss: 0.0092 - val_accuracy: 0.9961 - val_loss: 0.0182
Epoch 6/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9997 - loss: 0.0026 - val_accuracy: 0.9948 - val_loss: 0.0232
Epoch 7/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9987 - loss: 0.0046 - val_accuracy: 0.9961 - val_loss: 0.0201
Epoch 8/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9993 - loss: 0.0024 - val_accuracy: 0.9961 - val_loss:

In [6]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9906 - loss: 0.0244
Test accuracy: 0.987525999546051
